# Historical Validation using Meteo-France Best Track Data

Using CERF allocation, people affected and testing out Scenarios 2 and 3.

In [2]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [3]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely.geometry import LineString, Point
import os
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from src import constants

In [4]:
# loading all actual cyclone tracks
cyclone_tracks = pd.read_csv(
    Path(constants.AA_DATA_DIR)
    / "private"
    / "raw"
    / "moz"
    / "rsmc"
    / "data_cyclone_SWIO_19851986_to_20222023.csv"
)

In [5]:
cerf_emdat = pd.read_csv(
    Path(constants.AA_DATA_DIR)
    / "public/exploration/mdg/cerf_emdat_bngrc_data.csv"
)

In [6]:
cerf_emdat[cerf_emdat["Nom"] == "BATSIRAI"]

,Nom,Total Affected - EMDAT,Sinistres,CERF Allocations
56,BATSIRAI,112115.0,166671.0,4476918.0


In [7]:
cyclone_tracks["Lat"] = cyclone_tracks["Lat"].apply(
    lambda x: -x if x > 0 else x
)
cyclone_tracks.loc[:, "Season"] = cyclone_tracks.apply(
    lambda x: (
        f"{x['Year'] - 1}-{x['Year']}"
        if x["Month"] <= 6
        else f"{x['Year']}-{x['Year'] + 1}"
    ),
    axis=1,
)
# Ensure UTC is formatted as a two-digit hour
cyclone_tracks["UTC"] = cyclone_tracks["UTC"].apply(lambda x: f"{int(x):02}")
# Create a datetime column from separate date and time columns
cyclone_tracks["ISO_TIME"] = pd.to_datetime(
    cyclone_tracks[["Year", "Month", "Day", "UTC"]]
    .astype(str)
    .agg(" ".join, axis=1)
)

In [8]:
# Create 'geometry' column in cyclone_tracks GeoDataFrame using Lat/Lon for the points
cyclone_tracks["geometry"] = cyclone_tracks.apply(
    lambda row: Point(row["Lon"], row["Lat"]), axis=1
)

# Convert cyclone_tracks DataFrame to GeoDataFrame
cyclone_tracks_gdf = gpd.GeoDataFrame(
    cyclone_tracks, geometry="geometry", crs="EPSG:4326"
)
cyclone_tracks_buffer = cyclone_tracks.copy()
cyclone_tracks_buffer["RMW_km"] = cyclone_tracks_buffer["RMW (mn)"] * 1.852
cyclone_tracks_buffer["RMW_km"] = (
    cyclone_tracks_buffer["RMW_km"].fillna(1).replace(0, 1)
)
# Create 'geometry' column in cyclone_tracks_buffer GeoDataFrame
cyclone_tracks_buffer["geometry"] = cyclone_tracks_buffer.apply(
    lambda row: Point(row["Lon"], row["Lat"]), axis=1
)

# Convert cyclone_tracks_buffer DataFrame to GeoDataFrame
cyclone_tracks_gdf_buffer = gpd.GeoDataFrame(
    cyclone_tracks_buffer, geometry="geometry", crs="EPSG:4326"
)

# Create buffers for the cyclone tracks using the RMW_km column, converting km to degrees

# Create buffer around gdf_adm0 with the given distance
# Reproject to a CRS that uses meters
cyclone_tracks_gdf_buffer = cyclone_tracks_gdf_buffer.to_crs(
    epsg=constants.mdg_epsg
)

# Apply the buffer
cyclone_tracks_gdf_buffer["geometry"] = (
    cyclone_tracks_gdf_buffer.geometry.buffer(
        cyclone_tracks_gdf_buffer["RMW_km"] * 1000
    )
)

# Reproject back to the original CRS
cyclone_tracks_gdf_buffer = cyclone_tracks_gdf_buffer.to_crs(
    cyclone_tracks_gdf.crs
)

In [9]:
cyclone_tracks[cyclone_tracks["Name"] == "EMNATI"]["Max wind (kt)"].unique()

array([25., 30., 27., 33., 40., 43., 53., 60., 62., 70., 80., 85., 95.,
       78., 75., 50., 45., 48., 35., 20.])

In [10]:
cyclone_tracks[cyclone_tracks["Name"] == "BATSIRAI"]["Max wind (kt)"].unique()

array([ 20.,  22.,  23.,  25.,  28.,  30.,  40.,  50.,  60.,  85.,  nan,
        45.,  65.,  80.,  90.,  95.,  70., 100., 110.,  35.,  32.,  37.,
        43.,  47.,  42.])

In [11]:
adm0_path = (
    Path(constants.AA_DATA_DIR)
    / "public"
    / "raw"
    / "mdg"
    / "cod_ab"
    / "mdg_admbnda_adm0_BNGRC_OCHA_20181031.shp"
)
gdf_adm0 = gpd.read_file(adm0_path)

In [12]:
# Create buffer around gdf_adm0 with the given distance
# Reproject to a CRS that uses meters (e.g., EPSG:3857)
gdf_adm0_buffer = gdf_adm0.to_crs(epsg=constants.mdg_epsg)

# Apply the buffer of 100 km (100,000 meters)
gdf_adm0_buffer["geometry"] = gdf_adm0_buffer.geometry.buffer(
    constants.buffer * 1000
)  # 100 km = 100,000 meters

# Reproject back to the original CRS
gdf_adm0_buffer = gdf_adm0_buffer.to_crs(gdf_adm0.crs)

In [13]:
cyclone_tracks_sel = gpd.sjoin(
    cyclone_tracks_gdf_buffer,
    gdf_adm0_buffer,
    how="inner",
    predicate="intersects",
)
cyclone_tracks_sel = cyclone_tracks_sel.sort_values("ISO_TIME")
cyclone_tracks_sel_2006 = cyclone_tracks_sel[
    cyclone_tracks_sel["Year"] >= 2006
]

In [14]:
cyclone_tracks_sel_nobuff = gpd.sjoin(
    cyclone_tracks_gdf_buffer, gdf_adm0, how="inner", predicate="intersects"
)
cyclone_tracks_sel_nobuff = cyclone_tracks_sel_nobuff.sort_values("ISO_TIME")
cyclone_tracks_sel_2006_nobuff = cyclone_tracks_sel_nobuff[
    cyclone_tracks_sel_nobuff["Year"] >= 2006
]

In [15]:
cyclones_since_2006 = cyclone_tracks_sel_2006[
    cyclone_tracks_sel_2006["Max wind (kt)"] >= 48
]["Name"].unique()

In [ ]:
# which storms are met for Scenario 2 and 3
scenario2_storms = cyclone_tracks_sel_2006[
    cyclone_tracks_sel_2006["Max wind (kt)"] >= 64
]["Name"].unique()
scenario3_storms = cyclone_tracks_sel_2006[
    cyclone_tracks_sel_2006["Max wind (kt)"] >= 90
]["Name"].unique()
pd.DataFrame(scenario3_storms).to_csv(
    Path(
        constants.AA_DATA_DIR,
        "public",
        "exploration",
        "mdg",
        "scenario3_storms_mfr_best_track.csv",
    ),
    index=False,
    header=False,
)

In [16]:
# Scenario 2.5
scenario2_point_5_storms = np.union1d(
    cyclone_tracks_sel_2006_nobuff[
        cyclone_tracks_sel_2006_nobuff["Max wind (kt)"] >= 64
    ]["Name"].unique(),
    scenario3_storms,
)

In [17]:
cerf_emdat.columns

Index(['Nom', 'Total Affected - EMDAT', 'Sinistres', 'CERF Allocations'], dtype='object')

In [18]:
# adding year and season to output
cerf_emdat_df = cerf_emdat.merge(
    cyclone_tracks_sel_2006[["Name", "Year", "Season"]].drop_duplicates(),
    left_on="Nom",
    right_on="Name",
    how="right",
)

In [19]:
cerf_emdat_df = cerf_emdat_df[cerf_emdat_df["Name"].isin(cyclones_since_2006)]
cerf_emdat_df

,Nom,Total Affected - EMDAT,Sinistres,CERF Allocations,Name,Year,Season
1,BOLOETSE,NaN,6537.0,NaN,BOLOETSE,2006,2005-2006
3,BONDO,NaN,184998.0,NaN,BONDO,2006,2006-2007
4,CLOVIS,7313.0,NaN,NaN,CLOVIS,2007,2006-2007
6,NaN,NaN,NaN,NaN,FAVIO,2007,2006-2007
7,INDLALA,215198.0,1740911.0,1230903.0,INDLALA,2007,2006-2007
8,JAYA,NaN,74091.0,2200650.0,JAYA,2007,2006-2007
10,FAME,8613.0,17530.0,NaN,FAME,2008,2007-2008
11,IVAN,524153.0,487146.0,4625583.0,IVAN,2008,2007-2008
12,JOKWE,400.0,586.0,NaN,JOKWE,2008,2007-2008
15,FANELE,NaN,NaN,NaN,FANELE,2009,2008-2009


In [ ]:
cerf_emdat_df["Scenario 2"] = [
    storm in scenario2_storms for storm in cerf_emdat_df["Name"]
]
cerf_emdat_df["Scenario 3"] = [
    storm in scenario3_storms for storm in cerf_emdat_df["Name"]
]
cerf_emdat_df["Scenario 2.5"] = [
    storm in scenario2_point_5_storms for storm in cerf_emdat_df["Name"]
]
df = cerf_emdat_df[
    [
        "Name",
        "Season",
        "Scenario 2",
        # "Scenario 2.5",
        "Scenario 3",
        "Total Affected - EMDAT",
        "Sinistres",
        "CERF Allocations",
    ]
]

# Sort the DataFrame by 'Total Affected' in descending order
# Round values in 'Total Affected' and 'CERF Allocations' columns
df_sorted = df.sort_values(by="Total Affected - EMDAT", ascending=False)


# Define functions for highlighting and coloring bars
def highlight_true(val):
    color = "red" if val else ""
    return f"background-color: {color}"


def color_bar_affected(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, orange {val/df_sorted["Total Affected - EMDAT"].max()*100}%, transparent {val/df_sorted["Total Affected - EMDAT"].max()*100}%);'
    return ""


def color_bar_sinistres(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, #FFD700 {val/df_sorted["Sinistres"].max()*100}%, transparent {val/df_sorted["Sinistres"].max()*100}%);'
    return ""


def color_bar_cerf(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, green {val/df_sorted["CERF Allocations"].max()*100}%, transparent {val/df_sorted["CERF Allocations"].max()*100}%);'
    return ""


# Apply styling
styled_df = (
    df_sorted.style.map(
        highlight_true,
        subset=[
            "Scenario 2",
            # "Scenario 2.5",
            "Scenario 3",
        ],
    )
    .map(color_bar_affected, subset=["Total Affected - EMDAT"])
    .map(color_bar_sinistres, subset=["Sinistres"])
    .map(color_bar_cerf, subset=["CERF Allocations"])
    .format(
        {
            "Total Affected - EMDAT": lambda x: (
                f"{int(x):,}" if pd.notna(x) else ""
            ),  # Format with commas, no decimals, NaN as blank
            "Sinistres": lambda x: (
                f"{int(x):,}" if pd.notna(x) else ""
            ),  # Format with commas, no decimals, NaN as blank
            "CERF Allocations": lambda x: (
                f"{int(x):,}" if pd.notna(x) else ""
            ),  # Format with commas, no decimals, NaN as blank
        }
    )
    .set_table_styles(
        {"": [{"selector": "table", "props": "background-color: white;"}]}
    )
)

# Display the styled DataFrame
styled_df

,Name,Season,Scenario 2,Scenario 3,Total Affected - EMDAT,Sinistres,CERF Allocations
11,IVAN,2007-2008,True,True,"524,153","487,146","4,625,583"
42,ENAWO,2016-2017,True,True,"434,253","437,443","4,999,601"
63,FREDDY,2022-2023,True,False,"299,000","189,352","7,033,283"
28,GIOVANNA,2011-2012,True,False,"250,284","247,014","1,999,893"
7,INDLALA,2006-2007,True,True,"215,198","1,740,911","1,230,903"
21,HUBERT,2009-2010,False,False,"192,132",,
37,CHEDZA,2014-2015,False,False,"174,007",,
59,EMNATI,2021-2022,True,False,"169,000","172,178","1,470,268"
43,AVA,2017-2018,True,False,"161,318","161,328",
23,BINGIZA,2010-2011,True,False,"115,215","267,099",
